In [1]:
import os
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk-11.0.9.jdk/Contents/Home"
os.environ["SPARK_HOME"] = "/Users/allen/Documents/Msc/Semestre II/Big Data/material/spark-3.0.3-bin-hadoop2.7"

import pandas as pd
import findspark
findspark.init()

from pyspark.ml.linalg import Vectors
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

In [2]:
SpSession = SparkSession.builder.master("local[*]").getOrCreate()

21/12/15 17:43:59 WARN Utils: Your hostname, MacBook-Air-de-Allen.local resolves to a loopback address: 127.0.0.1; using 192.168.1.5 instead (on interface en0)
21/12/15 17:43:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/12/15 17:44:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/15 17:44:00 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [9]:
allDf = SpSession.read.parquet("./variables_finales.parquet")
allDf.limit(5).show()

+--------------------+---------+---------+---------------+----------------+----------------------+---------------+---------+---------------+----------------+----------------------+---------------+---------+---------------+----------------+----------------------+---------------+----------+----------------+-----------------+-----------------------+----------------+----------------+-----------------+-----------------------+----------------+-----------------+-----------------------+----------------+-----------------+-----------------------+--------------------+---------------------+---------------------------+---------+---------------+----------------+----------------------+---------------+---------+---------------+----------------+----------------------+---------------+----------+----------------+-----------------+-----------------------+----------------+----------------+-----------------+-----------------------+----------------+-----------------+-----------------------+------------------

In [11]:
# Eliminamos la columna origin del Dataframe
allDf = allDf.drop('origen')
allDf.limit(5).show()

+---------+---------+---------------+----------------+----------------------+---------------+---------+---------------+----------------+----------------------+---------------+---------+---------------+----------------+----------------------+---------------+----------+----------------+-----------------+-----------------------+----------------+----------------+-----------------+-----------------------+----------------+-----------------+-----------------------+----------------+-----------------+-----------------------+--------------------+---------------------+---------------------------+---------+---------------+----------------+----------------------+---------------+---------+---------------+----------------+----------------------+---------------+----------+----------------+-----------------+-----------------------+----------------+----------------+-----------------+-----------------------+----------------+-----------------+-----------------------+--------------------+------------------

### Calculamos la matriz de correlaciones

In [12]:
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=allDf.columns, outputCol=vector_col)
df_vector = assembler.transform(allDf).select(vector_col)
matrix = Correlation.corr(df_vector, vector_col)
cor_np = matrix.collect()[0][matrix.columns[0]].toArray()
cor_np

21/12/15 17:48:46 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/12/15 17:48:46 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


array([[ 1.        ,  0.27815427,  0.26970191, ..., -0.51054114,
        -0.13592374, -0.05250227],
       [ 0.27815427,  1.        ,  0.98677563, ..., -0.06840788,
         0.26913575,  0.17528191],
       [ 0.26970191,  0.98677563,  1.        , ..., -0.07108977,
         0.26035927,  0.17045259],
       ...,
       [-0.51054114, -0.06840788, -0.07108977, ...,  1.        ,
         0.60676584,  0.55113662],
       [-0.13592374,  0.26913575,  0.26035927, ...,  0.60676584,
         1.        ,  0.67553886],
       [-0.05250227,  0.17528191,  0.17045259, ...,  0.55113662,
         0.67553886,  1.        ]])

In [13]:
# Eliminamos columnas con alta correlación (1 de ellas)
high_corr_columns = ['mt_sumSeg','mt_sumLla','mt_sumSms','mt_nroDest','mo_sumSeg','mo_sumLla','mo_sumSms','mo_nroDest','mt_sumSmsService','mt_PropSegMobile','mt_PropSegService','mt_PropSegInternational','mt_PropLlaMobile','mt_PropLlaService','mt_PropLlaInternational','mt_PropSmsService','mt_PropNroDestMobile','mt_PropNroDestService','mo_PropSegMobile','mo_PropSegService','mo_PropSegInternational','mo_PropLlaMobile','mo_PropLlaService','mo_PropLlaInternational','mo_PropNroDestMobile','mo_PropNroDestService','mo_PropNroDestInternational']

allDf = allDf.drop(*high_corr_columns)
allDf.limit(5).toPandas().head()

,contactos,mt_sumSegMobile,mt_sumSegService,mt_sumSegInternational,mt_sumSegOthers,mt_sumLlaMobile,mt_sumLlaService,mt_sumLlaInternational,mt_sumLlaOthers,mt_sumSmsMobile,...,mo_sumLlaMobile,mo_sumLlaService,mo_sumLlaInternational,mo_sumLlaOthers,mo_nroDestMobile,mo_nroDestService,mo_nroDestInternational,mo_nroDestOthers,PropSegMobileTvsTotal,PropContactosTvsTotal
0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,0.0,0.0,2,2,0,0,0.00075,0.30000
1,2,0.0,0.0,0.0,56.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.99123,0.83333
2,9,447.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,...,10.0,0.0,0.0,1.0,6,0,0,1,0.41705,0.55000
3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,2,0,0,0,0.00336,0.16667
4,3,202.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.99754,0.87500


In [14]:
# allDF_final = pd.DataFrame(cor_np, index=allDf.columns,  columns=allDf.columns)
# allDF_final.to_csv('./allDF_final.csv')

### Kmeans con PySpark

In [14]:
# %pip install memory_profiler
%load_ext memory_profiler

In [15]:
def normalize(inRow):
    global bcMeans
    global bcStdDev
    
    meanArray = bcMeans.value
    stdArray = bcStdDev.value
    
    retArray = []
    for i in range(len(meanArray)):
        retArray.append((float(inRow[i])-float(meanArray[i]))/float(stdArray[i]))
        
    return Vectors.dense(retArray)

In [16]:
# Centrado y escalado 
summStats = allDf.describe().toPandas()
# print(summStats)
meanValues = summStats.iloc[1, 1:].values.tolist()
stdValues = summStats.iloc[2, 1:].values.tolist()

# Colocamos estos valores en una variable de difusion
bcMeans = SpSession.sparkContext.broadcast(meanValues)
bcStdDev = SpSession.sparkContext.broadcast(stdValues)

# Crear un marco de datos de Spark
allVectors = allDf.rdd.map(normalize)
allRows = allVectors.map(lambda f:Row(features=f))
allRowsDf = SpSession.createDataFrame(allRows)

In [17]:
from pyspark.ml.clustering import KMeans

modelPySpark = KMeans(k=5, seed=42)
%memit modelPySpark = modelPySpark.fit(allRowsDf)

peak memory: 2393.33 MiB, increment: -20.98 MiB


In [18]:
%memit predictions = modelPySpark.transform(allRowsDf)
predictions.show(10)

peak memory: 2399.31 MiB, increment: -73.62 MiB
+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[-0.1820677258988...|         2|
|[-0.6112321571936...|         0|
|[0.89084335233840...|         4|
|[-0.6112321571936...|         2|
|[-0.3966499415462...|         0|
|[1.74917221492816...|         3|
|[-0.3966499415462...|         2|
|[0.03251448974863...|         0|
|[1.53458999928072...|         2|
|[1.96375443057560...|         3|
+--------------------+----------+
only showing top 10 rows



### Kmeans con Pandas

In [27]:
import pandas as pd

column_names = ['contactos','mt_sumSegMobile','mt_sumSegService','mt_sumSegInternational','mt_sumSegOthers','mt_sumLlaMobile','mt_sumLlaService','mt_sumLlaInternational','mt_sumLlaOthers','mt_sumSmsMobile','mt_sumSmsInternational','mt_sumSmsOthers','mt_nroDestMobile','mt_nroDestService','mt_nroDestInternational','mt_nroDestOthers','mt_PropSmsMobile','mt_PropSmsInternational','mt_PropNroDestInternational','mo_sumSegMobile','mo_sumSegService','mo_sumSegInternational','mo_sumSegOthers','mo_sumLlaMobile','mo_sumLlaService','mo_sumLlaInternational','mo_sumLlaOthers','mo_nroDestMobile','mo_nroDestService','mo_nroDestInternational','mo_nroDestOthers','PropSegMobileTvsTotal','PropContactosTvsTotal']

finalColumns = [
    pd.read_csv("./allDf.csv/part-00000-7fcf4edf-f59b-42cd-b98c-29d69551f805-c000.csv", names=column_names),
    pd.read_csv("./allDf.csv/part-00001-7fcf4edf-f59b-42cd-b98c-29d69551f805-c000.csv", names=column_names),
    pd.read_csv("./allDf.csv/part-00002-7fcf4edf-f59b-42cd-b98c-29d69551f805-c000.csv", names=column_names),
    pd.read_csv("./allDf.csv/part-00003-7fcf4edf-f59b-42cd-b98c-29d69551f805-c000.csv", names=column_names),
    pd.read_csv("./allDf.csv/part-00004-7fcf4edf-f59b-42cd-b98c-29d69551f805-c000.csv", names=column_names)
]

X = pd.concat(finalColumns, axis=0, ignore_index=True)

In [29]:
X

,contactos,mt_sumSegMobile,mt_sumSegService,mt_sumSegInternational,mt_sumSegOthers,mt_sumLlaMobile,mt_sumLlaService,mt_sumLlaInternational,mt_sumLlaOthers,mt_sumSmsMobile,...,mo_sumLlaMobile,mo_sumLlaService,mo_sumLlaInternational,mo_sumLlaOthers,mo_nroDestMobile,mo_nroDestService,mo_nroDestInternational,mo_nroDestOthers,PropSegMobileTvsTotal,PropContactosTvsTotal
0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,0.0,0.0,2,2,0,0,0.00075,0.30000
1,2,0.0,0.0,0.0,56.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.99123,0.83333
2,9,447.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,...,10.0,0.0,0.0,1.0,6,0,0,1,0.41705,0.55000
3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,2,0,0,0,0.00336,0.16667
4,3,202.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.99754,0.87500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3761707,8,1728.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,90.0,...,8.0,0.0,0.0,0.0,4,0,0,0,0.78142,0.83333
3761708,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.50000,0.75000
3761709,3,255.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,3,0,0,0,0.66364,0.62500
3761710,20,691.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,4.0,...,19.0,0.0,0.0,1.0,11,0,0,1,0.37974,0.54762


In [30]:
from sklearn.cluster import KMeans

n_clusters = 5

skKmeansModel = KMeans(init="random", n_clusters=n_clusters, random_state=42)

%memit model = skKmeansModel.fit(X)

peak memory: 4875.59 MiB, increment: 2730.43 MiB


In [31]:
%memit predictions = model.predict(X)

peak memory: 3539.20 MiB, increment: 954.61 MiB


### Kmean con ray.util.joblib

In [ ]:
# # %pip install ray
# from sklearn.cluster import KMeans

# n_clusters = 5

# rayKmeansModel = KMeans(init="random", n_clusters=n_clusters, random_state=42)

# import joblib
# from ray.util.joblib import register_ray
# register_ray()

# with joblib.parallel_backend('ray'):
#     %memit model = rayKmeansModel.fit(X)

# %memit predictions = model.predict(X)